In [1]:
import datetime as dt
import math

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
# from tooling.enums import AssetClass, Index, Spot, StrikeSpread
# from tooling.fetch import fetch_option_data, fetch_spot_data
# from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
# bnf_pandas = pd.read_csv("../data/bnf_min.csv")
bnf_pandas = pd.read_csv("../data/ng_1hr.csv")
# bnf_pandas = pd.read_csv("../data/fin_min.csv")
# bnf_pandas = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# bnf_pandas = pd.read_csv("../data/bankex_min.csv")

In [3]:
# If Stocks Data ...
bnf_pandas["time"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["time"] = bnf_pandas["time"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["time"].dt.year >= 2017]
bnf_pandas['datetime'] = bnf_pandas['time']
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [4]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [5]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [6]:
bnf = bnf.rename({"open": "o", "high": "h", "low": "l", "close": "c"})

In [7]:
bnf.tail()

o,h,l,c,index,datetime
f64,f64,f64,f64,datetime[ns],datetime[ns]
324.2,329.8,323.1,327.0,2024-12-24 20:00:00,2024-12-24 20:00:00
327.0,332.5,327.0,331.4,2024-12-24 21:00:00,2024-12-24 21:00:00
331.3,332.8,326.6,332.4,2024-12-24 22:00:00,2024-12-24 22:00:00
332.2,336.9,329.6,334.1,2024-12-24 23:00:00,2024-12-24 23:00:00
328.9,328.9,327.3,327.7,2024-12-26 09:00:00,2024-12-26 09:00:00


In [84]:
def trade(df, x=0.02, y=0.01):
    # Ensure datetime column is in the correct format and set as index
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

    # Create a date column for grouping
    df['date'] = df.index.date
    tradebook = []

    # Iterate over each date for trade simulation
    for date, data in df.groupby('date'):
        if len(data) < 2:
            continue  # Skip if insufficient data

        # Extract 3 PM and 4 PM candles
        candle_3pm = data.between_time("19:00", "19:00")
        candle_4pm = data.between_time("20:00", "20:00")

        if candle_3pm.empty or candle_4pm.empty:
            continue  # Skip if 3 PM or 4 PM candles are missing
        
        # Get the high/low of 3 PM candle and 4 PM candle
        high_3pm, low_3pm = candle_3pm['h'].iloc[0], candle_3pm['l'].iloc[0]
        high_4pm, low_4pm = candle_4pm['h'].iloc[0], candle_4pm['l'].iloc[0]

        # Determine signal candle high/low and SL based on 3 PM or 4 PM conditions
        if high_4pm < high_3pm and low_4pm > low_3pm:  # 4 PM is an inside bar
            entry_price_long, entry_price_short = high_4pm, low_4pm
            sl_long, sl_short = low_4pm, high_4pm
        else:  # 3 PM is the signal candle
            entry_price_long, entry_price_short = high_3pm, low_3pm
            sl_long, sl_short = low_3pm, high_3pm

        # Define target prices based on entry and SL
        target_long = entry_price_long + 5 * (high_3pm - low_3pm)
        target_short = entry_price_short - 5 * (high_3pm - low_3pm)

        # Process trades after 4 PM
        trades = data[data.index >= pd.to_datetime(f"{date} 20:00:00")]
        
        # Initialize trade flags
        active_long, active_short = False, False
        long_trade_done, short_trade_done = False, False

        for idx, row in trades.iterrows():
            # Process long trade entry
            # print(idx, row)
            if not long_trade_done and row['h'] >= entry_price_long and not active_long:
                active_long = True
                long_trade_done = True
                entry_time, sl, target = idx, sl_long, target_long
                signal_candle_high = high_3pm if high_4pm > high_3pm else high_4pm
                signal_candle_low = low_3pm if low_4pm < low_3pm else low_4pm
                initial_sl_long = sl_long
                target_price = target_long
                # print(f'Long Entry Found at {idx}')

            if active_long:
                # Check exit conditions for long
                if row['l'] <= initial_sl_long:  # Stop-loss hit
                    exit_price, exit_remark, exit_time_long = initial_sl_long, 'SL', idx
                    active_long = False
                    # print(f'Long SL Found at {idx}')
                elif row['h'] >= target:  # Target hit
                    exit_price, exit_remark, exit_time_long = target, 'Target', idx
                    active_long = False
                    # print(f'Long Target Found at {idx}')
                elif idx == trades.index[-1]:  # End of the day exit
                    exit_price, exit_remark, exit_time_long = row['c'], 'EOD', idx
                    active_long = False
                    # print(f'Long EOD Found at {idx}')

                if not active_long:
                    points_captured = exit_price - entry_price_long
                    tradebook.append({
                        'entry_time': entry_time,
                        'exit_time': exit_time_long,
                        'entry_price': entry_price_long,
                        'exit_price': exit_price,
                        'points_captured': points_captured,
                        'remark': exit_remark,
                        'trade_type': 'Long',
                        'Signal Candle High': signal_candle_high,
                        'Signal Candle Low': signal_candle_low,
                        'Initial SL': initial_sl_long,
                        'Target Price': target_price,
                    })
                    

            # Process short trade entry
            if not short_trade_done and row['l'] <= entry_price_short and not active_short:
                active_short = True
                short_trade_done = True
                entry_time, sl, target = idx, sl_short, target_short
                signal_candle_high = high_3pm if high_4pm > high_3pm else high_4pm
                signal_candle_low = low_3pm if low_4pm < low_3pm else low_4pm
                initial_sl_short = sl_short
                target_price = target_short
                # print(f'Short Entry Found at {idx}')

            if active_short:
                # Check exit conditions for short
                if row['h'] >= initial_sl_short:  # Stop-loss hit
                    exit_price, exit_remark, exit_time_short = initial_sl_short, 'SL', idx
                    active_short = False
                    # print(f'Short SL Found at {idx}')
                elif row['l'] <= target:  # Target hit
                    exit_price, exit_remark, exit_time_short = target, 'Target', idx
                    active_short = False
                    # print(f'Short Target Found at {idx}')
                elif idx == trades.index[-1]:  # End of the day exit
                    exit_price, exit_remark, exit_time_short = row['c'], 'EOD', idx
                    active_short = False
                    # print(f'Short EOD Found at {idx}')

                if not active_short:
                    points_captured = entry_price_short - exit_price
                    tradebook.append({
                        'entry_time': entry_time,
                        'exit_time': exit_time_short,
                        'entry_price': entry_price_short,
                        'exit_price': exit_price,
                        'points_captured': points_captured,
                        'remark': exit_remark,
                        'trade_type': 'Short',
                        'Signal Candle High': signal_candle_high,
                        'Signal Candle Low': signal_candle_low,
                        'Initial SL': initial_sl_short,
                        'Target Price': target_price,
                    })

    # Return tradebook as DataFrame
    return pd.DataFrame(tradebook)


In [85]:
data = bnf
df = pd.DataFrame(data)
df.columns = ['o', 'h', 'l', 'c', 'index', 'datetime']
tradebook = trade(df, x=0.02, y=0.02)
# print(tradebook)

In [86]:
tradebook

,entry_time,exit_time,entry_price,exit_price,points_captured,remark,trade_type,Signal Candle High,Signal Candle Low,Initial SL,Target Price
0,2020-12-28 20:00:00,2020-12-28 23:00:00,167.8000,169.1000,1.3000,EOD,Long,167.8000,165.6000,165.0000,181.8000
1,2020-12-29 20:00:00,2020-12-29 23:00:00,177.4000,181.2000,3.8000,EOD,Long,177.4000,176.8000,175.1000,188.9000
2,2020-12-30 20:00:00,2020-12-30 23:00:00,178.6000,176.8000,1.8000,EOD,Short,181.1000,178.6000,181.3000,165.1000
3,2020-12-31 20:00:00,2020-12-31 23:00:00,181.5000,182.1000,-0.6000,EOD,Short,182.4000,181.5000,183.5000,171.5000
4,2021-01-04 20:00:00,2021-01-04 23:00:00,192.2000,189.3000,2.9000,EOD,Short,193.5000,192.2000,195.5000,175.7000
5,2021-01-05 20:00:00,2021-01-05 23:00:00,199.7000,197.9000,-1.8000,EOD,Long,199.7000,197.6000,196.6000,215.2000
6,2021-01-06 20:00:00,2021-01-06 23:00:00,196.7000,201.2000,4.5000,EOD,Long,196.7000,195.5000,194.7000,206.7000
7,2021-01-07 20:00:00,2021-01-07 23:00:00,200.4000,201.1000,0.7000,EOD,Long,200.4000,199.2000,197.6000,214.4000
8,2021-01-08 20:00:00,2021-01-08 23:00:00,196.0000,197.8000,1.8000,EOD,Long,196.0000,194.4000,193.9000,206.5000
9,2021-01-11 22:00:00,2021-01-11 22:00:00,193.2000,196.7000,-3.5000,SL,Short,195.3000,193.2000,196.7000,214.2000


In [87]:
tradebook['points_captured'].sum()

-1629.399999999999

In [88]:
tradebook.tail(30)

,entry_time,exit_time,entry_price,exit_price,points_captured,remark,trade_type,Signal Candle High,Signal Candle Low,Initial SL,Target Price
1463,2024-11-27 20:00:00,2024-11-27 20:00:00,274.3000,276.9000,-2.6000,SL,Short,276.9000,274.3000,276.9000,243.3000
1464,2024-11-28 21:00:00,2024-11-28 21:00:00,273.0000,275.7000,-2.7000,SL,Short,274.9000,273.0000,275.7000,289.2000
1465,2024-11-28 21:00:00,2024-11-28 23:00:00,275.7000,278.7000,3.0000,EOD,Long,274.9000,273.0000,273.0000,289.2000
1466,2024-11-29 20:00:00,2024-11-29 21:00:00,282.9000,280.4000,-2.5000,SL,Long,282.9000,280.6000,280.4000,295.4000
1467,2024-11-29 21:00:00,2024-11-29 21:00:00,280.4000,282.9000,-2.5000,SL,Short,282.9000,280.6000,282.9000,267.9000
1468,2024-12-02 20:00:00,2024-12-02 23:00:00,273.9000,273.2000,-0.7000,EOD,Long,273.9000,273.3000,268.4000,301.4000
1469,2024-12-03 20:00:00,2024-12-03 23:00:00,264.6000,258.2000,6.4000,EOD,Short,266.4000,264.6000,267.6000,249.6000
1470,2024-12-04 20:00:00,2024-12-04 21:00:00,258.5000,254.7000,-3.8000,SL,Long,258.5000,254.8000,254.7000,277.5000
1471,2024-12-04 21:00:00,2024-12-04 23:00:00,254.7000,258.5000,-3.8000,SL,Short,258.5000,254.8000,258.5000,235.7000
1472,2024-12-05 20:00:00,2024-12-05 22:00:00,265.5000,262.6000,-2.9000,SL,Long,265.5000,262.9000,262.6000,280.0000
